## Importing Libraries

In [28]:
import os
import argparse
import logging
import yaml
from string import Template
import json 


import kfp
from kfp import dsl
from kfp.components import func_to_container_op

## Define Kubeflow Pipeline with Inference Service and then create service

In [66]:
@func_to_container_op
def show_results(accuracy :float) -> None:
    print(f"Accuracy of neural network: {accuracy}")

@dsl.pipeline(name='demo_pipeline', description='Fits a simple neural network')
def first_pipeline():

    download = kfp.components.load_component_from_file('download_data/download_data.yaml')
    preprocess_data = kfp.components.load_component_from_file('preprocess_data/preprocess_data.yaml')
    train_model = kfp.components.load_component_from_file('train_model/train_model.yaml')

    download_task = download()
    preprocess_data_task = preprocess_data(download_task.output)
    train_model_task = train_model(preprocess_data_task.output)
    
    show_results(train_model_task.outputs['Accuracy'])
    
    with open('./inference_service/inference_service.yaml', 'r') as f:
        doc = yaml.safe_load(f)

    template = Template(str(doc))
    tensorjson = template.substitute({'NAMESPACE': "jarmi-recio"})
    deployment = json.loads(tensorjson.replace("'", '"'))
    print(deployment)

    deploy = dsl.ResourceOp(
        name="demo-inference-service",
        k8s_resource=deployment,
        action="apply")
    
    deploy.after(train_model_task)
if __name__ == '__main__':
    file_name = 'pipeline.yaml'
    kfp.compiler.Compiler().compile(first_pipeline, file_name)

{'apiVersion': 'serving.kserve.io/v1beta1', 'kind': 'InferenceService', 'metadata': {'labels': {'controller-tools.k8s.io': '1.0'}, 'name': 'kserve-demo', 'namespace': 'jarmi-recio', 'annotations': {'sidecar.istio.io/inject': 'false'}}, 'spec': {'predictor': {'minReplicas': 1, 'containers': {'name': 'kserve-container', 'image': 'jarmirecio/inference_service_kserve', 'imagePullPolicy': 'Always'}}}}


## Define Kubeflow Inference Service and then create service

In [93]:
@func_to_container_op
def show_results(accuracy :float) -> None:
    print(f"Accuracy of neural network: {accuracy}")

@dsl.pipeline(name='demo_pipeline', description='Fits a simple neural network')
def first_pipeline():

    with open('./inference_service/inference_service.yaml', 'r') as f:
        doc = yaml.safe_load(f)
    
    template = Template(str(doc))
    tensorjson = template.substitute({'NAMESPACE': "jarmi-recio"})
    deployment = json.loads(tensorjson.replace("'", '"'))
    print(deployment)

    deploy = dsl.ResourceOp(
        name="demo-inference-service",
        k8s_resource=deployment,
        action="apply")

if __name__ == '__main__':
    file_name = 'pipeline.yaml'
    kfp.compiler.Compiler().compile(first_pipeline, file_name)

{'apiVersion': 'serving.kserve.io/v1beta1', 'kind': 'InferenceService', 'metadata': {'labels': {'controller-tools.k8s.io': '1.0'}, 'name': 'kserve-demo', 'namespace': 'jarmi-recio', 'annotations': {'sidecar.istio.io/inject': 'false'}}, 'spec': {'predictor': {'containers': [{'name': 'kserve-container', 'image': 'jarmirecio/inference_service_kserve_0_7_0'}]}}}


## Upload pipeline

In [96]:
import kfp
client=kfp.Client()
filepath = '/home/jovyan/'+file_name
name= 'kubeflow_demo_inference_kserve_20'
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)